In [19]:
import itertools

def generate_grids(grid, max_replacements):
    # Find indices of 0s in the grid
    zero_indices = [(i, j) for i, row in enumerate(grid) for j, val in enumerate(row) if val == 0]
    
    # Generate all possible combinations of replacements
    for r in range(min(max_replacements, len(zero_indices)) + 1):
        for indices in itertools.combinations(zero_indices, r):
            for replacements in itertools.product(range(1, 15), repeat=r):
                new_grid = [row[:] for row in grid]  # Create a deep copy of the grid
                for (i, j), value in zip(indices, replacements):
                    new_grid[i][j] = value
                yield new_grid

In [20]:
initial_grid = [[5, 15, 15], [0, 0, 0], [15, 15, 0], [0, 0, 0], [0, 15, 15], [0, 0, 6]]
max_replacements = 2

results = list(generate_grids(initial_grid, max_replacements))
print(len(results))  #

8961


In [21]:
# brutce force
from grid import Grid
from cart import Cart
import os
from ffmpeg import FFmpeg
import json
from utils import DIRECTION
import copy
import shutil
from tqdm import tqdm
import numpy as np
import time

root_folder = "blog_output"
os.makedirs(root_folder, exist_ok=True)


grid_to_visual = [
    "levels/1-1.json",
    "levels/1-3.json",
    "levels/1-6.json",
    "levels/1-7.json",
    "levels/1-8.json",
    "levels/1-9.json",
]

for grid_file in grid_to_visual:
    name = os.path.basename(grid_file).split(".")[0]
    output_folder = os.path.join(root_folder, name)
    # os.makedirs(output_folder, exist_ok=True)

    with open(grid_file, "r") as f:
        data = json.load(f)
    grid = data["grid"]
    destination = data["destination"]
    max_tracks = data["max_tracks"]
    carts = [
        Cart((cart["x"], cart["y"]), DIRECTION(cart["direction"]), cart["order"])
        for cart in data["carts"]
    ]
    frames = []
    start_time = time.time()
    for i, _g in enumerate(tqdm(generate_grids(grid, max_tracks), desc=name + " ")):
        g = Grid(_g, destination)
        frames.append(g)
        if not g.is_grid_valid():
            continue
        carts_copy = [copy.deepcopy(cart) for cart in carts]
        simulate_result = g.simulate(carts_copy)
        if simulate_result[0] == "success":
            break
    print(f"Time taken for {name}: {time.time() - start_time}")
    print(f"Total frames: {i}")
    os.makedirs(output_folder, exist_ok=True)
    total_frames = 200
    indices = np.round(np.linspace(0, i-1, total_frames)).astype(int)
    for i, j in enumerate(indices):
        img = frames[j].get_image()
        img.save(os.path.join(output_folder, f"{name}_{i}.png"))
    # add 10 more frames to make the video more stable
    for k in range(i, i+10):
        img = frames[-1].get_image()
        img.save(os.path.join(output_folder, f"{name}_{k}.png"))
    full_path = os.getcwd()
    ffmpeg = (
        FFmpeg()
        .option("y")
        .input(os.path.join(output_folder, f"{name}_%d.png"), framerate=20)
        .output(
            os.path.join(full_path, root_folder, f"{name}.mp4"),
            {
                "c:v": "libx265",
            },
        )
    )
    out=ffmpeg.execute()
    shutil.rmtree(output_folder)

1-1 : 1475it [00:00, 119712.04it/s]

Time taken for 1-1: 0.014321327209472656
Total frames: 1475



1-3 : 37it [00:00, 36932.23it/s]


Time taken for 1-3: 0.001999378204345703
Total frames: 37


1-6 : 518467it [00:04, 111340.69it/s]


...

Time taken for 1-7: 0.37085723876953125
Total frames: 56017


1-8 : 76557it [00:00, 126599.86it/s]


Time taken for 1-8: 0.6057186126708984
Total frames: 76557


1-9 : 5078095it [01:04, 78929.27it/s] 


Time taken for 1-9: 64.33928227424622
Total frames: 5078094


In [28]:
from collections import deque

for grid_file in grid_to_visual:
    name = os.path.basename(grid_file).split(".")[0]
    output_folder = os.path.join(root_folder, name)
    os.makedirs(output_folder, exist_ok=True)

    with open(grid_file, "r") as f:
        data = json.load(f)
    grid = data["grid"]
    destination = data["destination"]
    max_tracks = data["max_tracks"]
    g = Grid(grid, destination)
    carts = [
        Cart((cart["x"], cart["y"]), DIRECTION(cart["direction"]), cart["order"])
        for cart in data["carts"]
    ]
    queue = deque([(g, carts)])
    frames = []
    start_time = time.time()
    while len(queue):
        g, carts = queue.pop()
        if g.tile_placed_count > max_tracks:
            continue
        g = copy.deepcopy(g)
        carts = copy.deepcopy(carts)
        result = g.simulate(carts, max_iter=10)
        frames.append(g)
        if result[0] == "empty_pos_reached":
            possible_grids = g.get_possible_grid(result[1])
            for grid in possible_grids:
                queue.append((grid, carts))
        if result[0] == "success":
            break
    print(f"Time taken for {name}: {time.time() - start_time}")
    print(f"Total frames: {len(frames)}")
    total_frames = 50
    indices = np.round(np.linspace(0, len(frames)-1, total_frames)).astype(int)
    for i, j in enumerate(indices):
        img = frames[j].get_image()
        img.save(os.path.join(output_folder, f"{name}_{i}.png"))
    # add 10 more frames to make the video more stable
    for k in range(i, i+10):
        img = frames[-1].get_image()
        img.save(os.path.join(output_folder, f"{name}_{k}.png"))
    full_path = os.getcwd()
    ffmpeg = (
        FFmpeg()
        .option("y")
        .input(os.path.join(output_folder, f"{name}_%d.png"), framerate=20)
        .output(
            os.path.join(full_path, root_folder, f"{name}_optimize.mp4"),
            {
                "c:v": "libx265",
            },
        )
    )
    ffmpeg.execute()
    shutil.rmtree(output_folder)

Time taken for 1-1: 0.0009999275207519531
Total frames: 4
Time taken for 1-3: 0.0
Total frames: 3
Time taken for 1-6: 0.0051593780517578125
Total frames: 41
Time taken for 1-7: 0.0
Total frames: 4
Time taken for 1-8: 0.001008749008178711
Total frames: 6
